In [16]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_rows', 8)

path = 'ppdai_3_23'

In [17]:
loan_characteristics = pd.read_csv(path + '/LC.csv')
loan_characteristics = loan_characteristics.rename(columns={
        '初始评级': 'initialGrade', '性别': 'sex', "借款期限": "totalPeriod","历史逾期还款期数":"overDuePeriod",
    "历史正常还款期数": "repayment", "借款金额": "money", "年龄": "age", "手机认证": "phone", "户口认证": "census", "视频认证": "video",
    "学历认证": "edu", "征信认证": "credit", "淘宝认证": "taobao"})
loan_characteristics.replace(to_replace="成功认证", value='Y',inplace=True, regex=False)
loan_characteristics.replace(to_replace="未成功认证", value='N',inplace=True, regex=False)

loan_periodic = pd.read_csv(path + '/LP.csv')
loan_periodic = loan_periodic.rename(columns={'到期日期': 'dueDate', '还款日期': 'payDate', '还款状态': 'status'})
display(loan_characteristics.shape, loan_periodic.shape)

(328553, 21)

(3203276, 10)

In [19]:
# calc overDue for LP, save it into file
import datetime

def calc_over_due(x):
    dueDate = x['dueDate']
    payDate = x['payDate']
    
    over_due = -9999

    if x['status'] != 0:
        overDue = datetime.datetime.strptime(payDate, '%Y-%m-%d') - datetime.datetime.strptime(dueDate, '%Y-%m-%d')
        over_due = overDue.days

    return over_due

def saveToFile():
    loan_periodic['overDue'] = loan_periodic.apply(calc_over_due, axis=1)
    loan_periodic.to_csv(path + '/LP_handle.csv', encoding="utf-8")


loan_periodic = pd.read_csv(path + '/LP_handle.csv')
display(loan_periodic.head(2))

,Unnamed: 0,ListingId,期数,status,应还本金,应还利息,剩余本金,剩余利息,dueDate,payDate,recorddate,overDue
0,0,126541,1,1,1380.23,270.00,0.0,0.0,2015-06-04,2015-06-04,2017-02-22,0
1,1,126541,2,1,1400.94,249.29,0.0,0.0,2015-07-04,2015-07-04,2017-02-22,0


In [21]:
#  calc maxOverDue for LC, save it to file
def saveMaxOverDueToFile():
    loan_periodic_overDue_greatEqual_0 = loan_periodic[(loan_periodic.overDue >= 0)]
    loan_all = pd.merge(loan_periodic_overDue_greatEqual_0, loan_characteristics)
    loan_all["maxOverDue"] = loan_all.groupby("ListingId")["overDue"].transform(max)
    pd_maxOverDue = loan_all[["ListingId", "maxOverDue"]].drop_duplicates();
    pd_temp1 = pd.merge(loan_characteristics, pd_maxOverDue, "outer")
    pd_temp1.to_csv(path + '/LC_handle.csv', encoding='utf-8');
    display(pd_temp1.head(1))

saveMaxOverDueToFile()

,ListingId,money,totalPeriod,借款利率,借款成功日期,initialGrade,借款类型,是否首标,age,sex,phone,census,video,edu,credit,taobao,历史成功借款次数,历史成功借款金额,总待还本金,repayment,overDuePeriod,maxOverDue
0,126541,18000,12,18.0,2015-05-04,C,其他,否,35,男,Y,N,Y,N,N,N,11,40326.0,8712.73,57,16,2.0


In [ ]:
loan_characteristics_ab = loan_characteristics[(loan_characteristics.initialGrade == "B") | (loan_characteristics.initialGrade == "A")]
loan_main = pd.merge(loan_characteristics, loan_periodic, on='ListingId')
loan_main_ab = loan_main[(loan_main.initialGrade == "B") | (loan_main.initialGrade == "A")]
display(loan_main_ab.head(2))

In [9]:
print(loan_characteristics_ab.shape)
print(loan_main_ab.shape)
print(loan_periodic.shape)
display(loan_periodic.head(1))

(43472, 21)
(375440, 30)
(3203276, 10)


,ListingId,期数,status,应还本金,应还利息,剩余本金,剩余利息,dueDate,payDate,recorddate
0,126541,1,1,1380.23,270.0,0.0,0.0,2015-06-04,2015-06-04,2017-02-22


In [10]:
loan_periodic_filter = loan_periodic[loan_periodic.payDate != "\\N"]
loan_periodic_filter['payDate'] = pd.to_datetime(loan_periodic_filter['payDate'])
loan_periodic_filter['dueDate'] = pd.to_datetime(loan_periodic_filter['dueDate'])
loan_periodic_filter['overDue'] = loan_periodic_filter['payDate'].sub(loan_periodic_filter['dueDate'])
display(loan_periodic_filter.head(10))
display(loan_periodic_filter[loan_periodic_filter['overDue'] > "15 days"])
#loan_periodic["overDue"] = pd.to_datetime(loan_periodic["payDate"]).subtract(pd.to_datetime(loan_periodic["dueDate"]), fill_value=0)

d:\program files\python35\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
d:\program files\python35\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
d:\program files\python35\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,ListingId,期数,status,应还本金,应还利息,剩余本金,剩余利息,dueDate,payDate,recorddate,overDue
0,126541,1,1,1380.23,270.00,0.0,0.0,2015-06-04,2015-06-04,2017-02-22,0 days
1,126541,2,1,1400.94,249.29,0.0,0.0,2015-07-04,2015-07-04,2017-02-22,0 days
2,126541,3,1,1421.95,228.28,0.0,0.0,2015-08-04,2015-08-04,2017-02-22,0 days
3,126541,4,1,1443.28,206.95,0.0,0.0,2015-09-04,2015-09-04,2017-02-22,0 days
4,126541,5,1,1464.93,185.30,0.0,0.0,2015-10-04,2015-10-04,2017-02-22,0 days
5,126541,6,1,1486.90,163.33,0.0,0.0,2015-11-04,2015-11-04,2017-02-22,0 days
6,126541,7,2,1509.21,141.02,0.0,0.0,2015-12-04,2015-12-05,2017-02-22,1 days
7,126541,8,1,1531.85,118.38,0.0,0.0,2016-01-04,2016-01-04,2017-02-22,0 days
8,126541,9,2,1554.82,95.41,0.0,0.0,2016-02-04,2016-02-05,2017-02-22,1 days
9,126541,10,2,1578.15,72.08,0.0,0.0,2016-03-04,2016-03-06,2017-02-22,2 days


,ListingId,期数,status,应还本金,应还利息,剩余本金,剩余利息,dueDate,payDate,recorddate,overDue
200,1536991,10,2,346.18,21.18,0.00,0.00,2015-11-04,2015-12-20,2017-02-22,46 days
201,1536991,11,2,353.10,14.26,0.00,0.00,2015-12-04,2015-12-21,2017-02-22,17 days
245,1538211,7,2,251.67,26.23,0.00,0.00,2015-08-04,2015-09-10,2017-02-22,37 days
283,1541761,6,2,825.13,101.21,0.00,0.00,2015-07-04,2015-08-01,2017-02-22,28 days
284,1541761,7,2,838.88,87.46,0.00,0.00,2015-08-04,2015-10-03,2017-02-22,60 days
285,1541761,8,2,852.86,73.48,0.00,0.00,2015-09-04,2015-10-03,2017-02-22,29 days
287,1541761,10,2,881.53,44.81,0.00,0.00,2015-11-04,2016-03-19,2017-02-22,136 days
336,1547751,11,2,353.22,13.67,0.00,0.00,2015-12-04,2016-01-10,2017-02-22,37 days
569,1563891,9,2,303.48,20.74,0.00,0.00,2015-10-02,2015-11-18,2017-02-22,47 days
570,1563891,10,2,308.54,15.68,0.00,0.00,2015-11-02,2015-11-25,2017-02-22,23 days
